In [1]:
import pandas as pd
import numpy as np

In [2]:
pid_traindata = pd.read_csv("pid_traindata_sample.csv")

In [3]:
LLC_FEATURES = [
    'pitch-deg',  # 飞机俯仰角
    'roll-deg',  # 飞机滚转角
    'heading-deg',  # 飞机朝向
    'vsi-fpm',  # 爬升速度
    'uBody-fps',  # 飞机沿机身X轴的速度
    'vBody-fps',  # 飞机沿机身Y轴的速度
    'wBody-fps',  # 飞机沿机身Z轴的速度
    'x-accel-fps_sec',  # 飞机沿机身X轴的加速度
    'y-accel-fps_sec',  # 飞机沿机身Y轴的加速度
    'z-accel-fps_sec',  # 飞机沿机身z轴的加速度
]


In [4]:
DATA_BOUNDS = {
    'pitch-deg': [-90., 90.],  # 飞机俯仰角
    'roll-deg': [-180., 180.],  # 飞机滚转角
    'heading-deg': [0., 360.],  # 飞机朝向
    'vsi-fpm': [0., 10.0],  # 爬升速度
    'uBody-fps': [0., 600.],  # 飞机沿机身X轴的速度
    'vBody-fps': [-200., 200.],  # 飞机沿机身Y轴的速度
    'wBody-fps': [-200., 200.],  # 飞机沿机身Z轴的速度
    'x-accel-fps_sec': [0., 50.],  # 飞机沿机身X轴的加速度
    'y-accel-fps_sec': [-30., 30.],  # 飞机沿机身Y轴的加速度
    'z-accel-fps_sec': [-300., 300.],  # 飞机沿机身z轴的加速度
    'aileron': [-1, 1],  # 副翼 控制飞机翻滚 [-1,1] left/right
    'elevator': [-1, 1],  # 升降舵 控制飞机爬升 [-1,1] up/down
    'rudder': [-1, 1],  # 方向舵 控制飞机转弯（地面飞机方向控制） 0 /enter
    'throttle0': [0, 1],  # 油门0
    'throttle1': [0, 1],  # 油门1
    'flaps': [0, 1], # 襟翼 在飞机起降过程中增加升力，阻力  Key[ / ]	Extend / retract flaps
    #TODO: 方向舵调整片
    'a_aileron': [-1, 1],  # 副翼 控制飞机翻滚 [-1,1] left/right
    'a_elevator': [-1, 1],  # 升降舵 控制飞机爬升 [-1,1] up/down
    'a_rudder': [-1, 1],  # 方向舵 控制飞机转弯（地面飞机方向控制） 0 /enter
    'a_throttle0': [0, 1],  # 油门0
    'a_throttle1': [0, 1],  # 油门1
}

In [5]:
data_bounds = pd.DataFrame.from_dict(DATA_BOUNDS)

In [6]:
data_bounds

,pitch-deg,roll-deg,heading-deg,vsi-fpm,uBody-fps,vBody-fps,wBody-fps,x-accel-fps_sec,y-accel-fps_sec,z-accel-fps_sec,...,elevator,rudder,throttle0,throttle1,flaps,a_aileron,a_elevator,a_rudder,a_throttle0,a_throttle1
0,-90.0,-180.0,0.0,0.0,0.0,-200.0,-200.0,0.0,-30.0,-300.0,...,-1,-1,0,0,0,-1,-1,-1,0,0
1,90.0,180.0,360.0,10.0,600.0,200.0,200.0,50.0,30.0,300.0,...,1,1,1,1,1,1,1,1,1,1


In [7]:
(pid_traindata - data_bounds.min()).min()

a_aileron                0.848693
a_elevator               0.406004
a_rudder                 0.907000
a_throttle0              0.010000
a_throttle1              0.010000
ai-offset                     NaN
ai-pitch                      NaN
ai-roll                       NaN
aileron                  0.848693
airspeed-kt                   NaN
alt-ft                        NaN
altitude                      NaN
crashed                       NaN
down-accel-fps_sec            NaN
east-accel-fps_sec            NaN
elevator                 0.406004
flaps                    0.000000
heading-deg             89.992182
hi-heading                    NaN
latitude                      NaN
longitude                     NaN
north-accel-fps_sec           NaN
pitch-deg               90.372688
roll-deg               178.511568
rudder                   0.907000
speed-down-fps                NaN
speed-east-fps                NaN
speed-north-fps               NaN
throttle0                0.000000
throttle1     

In [8]:
(pid_traindata - data_bounds.min()).max()

a_aileron                1.148843
a_elevator               1.090589
a_rudder                 1.035000
a_throttle0              0.600000
a_throttle1              0.600000
ai-offset                     NaN
ai-pitch                      NaN
ai-roll                       NaN
aileron                  1.148843
airspeed-kt                   NaN
alt-ft                        NaN
altitude                      NaN
crashed                       NaN
down-accel-fps_sec            NaN
east-accel-fps_sec            NaN
elevator                 1.090589
flaps                    0.000000
heading-deg             93.015763
hi-heading                    NaN
latitude                      NaN
longitude                     NaN
north-accel-fps_sec           NaN
pitch-deg              113.129216
roll-deg               181.513065
rudder                   1.035000
speed-down-fps                NaN
speed-east-fps                NaN
speed-north-fps               NaN
throttle0                0.600000
throttle1     

In [9]:
res = ((pid_traindata - data_bounds.min())/(data_bounds.max() - data_bounds.min())).dropna(axis=1,how="any")

In [10]:
info= pd.concat([res.min(),res.max()], axis=1)
np.array(info[1])

array([0.5744216 , 0.54529429, 0.5175    , 0.6       , 0.6       ,
       0.5744215 , 0.5452945 , 0.        , 0.25837712, 0.62849564,
       0.50420296, 0.5175    , 0.6       , 0.6       , 0.7273467 ,
       0.50000147, 0.        , 0.65314495, 0.29801502, 0.56153998,
       0.46315494])

In [31]:
state = info.T.loc[1,LLC_FEATURES] - pid_traindata.loc[0]
state

a_aileron                    NaN
a_elevator                   NaN
a_rudder                     NaN
a_throttle0                  NaN
a_throttle1                  NaN
ai-offset                    NaN
ai-pitch                     NaN
ai-roll                      NaN
aileron                      NaN
airspeed-kt                  NaN
alt-ft                       NaN
altitude                     NaN
crashed                      NaN
down-accel-fps_sec           NaN
east-accel-fps_sec           NaN
elevator                     NaN
flaps                        NaN
fly_mode                     NaN
heading-deg           -89.741623
hi-heading                   NaN
latitude                     NaN
longitude                    NaN
north-accel-fps_sec          NaN
pitch-deg               0.204496
roll-deg                0.504203
rudder                       NaN
speed-down-fps               NaN
speed-east-fps               NaN
speed-north-fps              NaN
throttle0                    NaN
throttle1 

In [12]:
tmp=(res- res.min())**2

In [13]:
tmp.sum(axis=1).idxmin()

0

In [14]:
res.loc[0]

a_aileron          0.500000
a_elevator         0.500000
a_rudder           0.500000
a_throttle0        0.010000
a_throttle1        0.010000
aileron            0.500000
elevator           0.500000
flaps              0.000000
heading-deg        0.250000
pitch-deg          0.502356
roll-deg           0.500000
rudder             0.500000
throttle0          0.000000
throttle1          0.000000
uBody-fps          0.000000
vBody-fps          0.500000
vsi-fpm            0.000000
wBody-fps          0.500000
x-accel-fps_sec    0.000000
y-accel-fps_sec    0.500000
z-accel-fps_sec    0.446667
Name: 0, dtype: float64

In [21]:
pid_traindata["fly_mode"] = (pid_traindata.loc[:,["altitude"]] > 23).astype(int) + (pid_traindata.loc[:,["altitude"]] > 6000).astype(int)

In [23]:
groups = pid_traindata.groupby("fly_mode")

In [30]:
for tag,group in groups:
    print(group)

      aileron  elevator  rudder  flaps  throttle0  throttle1  vsi-fpm  \
0    0.000000  0.000000   0.000    0.0       0.00       0.00      0.0   
1    0.000000  0.000000   0.000    0.0       0.00       0.00      0.0   
2    0.000000  0.000000   0.000    0.0       0.01       0.01      0.0   
3    0.000000  0.000000   0.000    0.0       0.01       0.01      0.0   
4    0.000000  0.000000   0.000    0.0       0.02       0.02      0.0   
5    0.000000  0.000000   0.000    0.0       0.02       0.02      0.0   
6    0.000000  0.000000   0.000    0.0       0.03       0.03      0.0   
7    0.000000  0.000000   0.000    0.0       0.03       0.03      0.0   
8    0.000000  0.000000   0.000    0.0       0.04       0.04      0.0   
9    0.000000  0.000000   0.000    0.0       0.04       0.04      0.0   
10   0.000000  0.000000   0.000    0.0       0.05       0.05      0.0   
11   0.000000  0.000000   0.000    0.0       0.05       0.05      0.0   
12   0.000000  0.000000   0.000    0.0       0.06  